In [1]:
import requests
import bs4 as bs
import lxml

httpaddr = "https://www.clarin.com"

requests_ret = requests.get(httpaddr)
print(requests_ret, end = ' || ')
soup = bs.BeautifulSoup(requests_ret.text, 'lxml')
# Notas:
# Everything semmantically a header, is tagged with <h2>. To all scrapping practical
# purposes, all headers except h2 are deemded irrelevant
articles = soup.find_all("article"); print("articles :", len(articles), end = ' / ')
sections = soup.find_all("section"); print("sections :", len(sections), end = ' / ')
headers = soup.find_all("header"); print("headers :", len(headers), end = ' / ')
h1s = soup.find_all("h1"); print("h1s :", len(h1s), end = ' / ')
h2s = soup.find_all("h2"); print("h2s :", len(h2s), end = ' / ')
h3s = soup.find_all("h3"); print("h3s :", len(h3s), end = ' / ')
h4s = soup.find_all("h4"); print("h4s :", len(h4s), end = ' / ')
h5s = soup.find_all("h5"); print("h5s :", len(h5s), end = ' / ')
h6s = soup.find_all("h6"); print("h6s :", len(h6s))
# only <h2>s hold relevance
# the only <h1> holds an SVG graphic

for discard_tag in ("script", "style"):
    for t in soup.find_all(discard_tag): t.extract()

<Response [200]> || articles : 106 / sections : 18 / headers : 13 / h1s : 1 / h2s : 136 / h3s : 0 / h4s : 0 / h5s : 0 / h6s : 0


In [2]:
for i, header in enumerate(headers):
    print(f'header [{i}]', len(f'header [{i}]') * '-', sep = '\n')
    for j, h2 in enumerate(header.find_all('h2')):
        print(f'\t<h2>[{h2s.index(h2)}]:', h2)
        print()

header [0]
----------
	<h2>[0]: <h2><a href="https://www.clarin.com/economia/dolar-blue-hoy-cotiza-jueves-20-octubre_0_ES5uZbgum5.html" onclick="metricsView('bloques_home','bloque_noticias_hoy|ES5uZbgum5', true);" target="_self">Dólar blue</a></h2>

	<h2>[1]: <h2><a href="https://www.clarin.com/politica/cristina-kirchner-probable-revolucion-federal-participado-intento-atentado-_0_mBl9BZWgOP.html" onclick="metricsView('bloques_home','bloque_noticias_hoy|mBl9BZWgOP', true);" target="_self">Cristina Kirchner</a></h2>

	<h2>[2]: <h2><a href="https://www.clarin.com/politica/cambios-presupuesto-podra-descontar-ganancias-40-gastos-escolares_0_hwX5mlgd6d.html" onclick="metricsView('bloques_home','bloque_noticias_hoy|hwX5mlgd6d', true);" target="_self">Ganancias</a></h2>

	<h2>[3]: <h2><a href="https://www.clarin.com/economia/gobierno-oficializo-refuerzo-alimentario-adultos-ingresos-45-000-podran-acceder_0_hkDHSxhqKH.html" onclick="metricsView('bloques_home','bloque_noticias_hoy|hkDHSxhqKH', tr

In [3]:
def tag_nearest_attrs(tag, attr):
    cursor = tag
    while cursor:
        attrs = [tag for tag in cursor.find_all(lambda x: x.get(attr)) if tag]
        if attrs:
            return attrs
        else:
            cursor = cursor.parent
    else:
        return None

def tag_nearest_attr_val(tag, attr, val):
    """
    Given a tag, walks upwards the hierarchy and stops when reaching a tag
    with at least one tag in its descendants hierarchy having attr attribute
    with value val, and returns a list of the nodes with that attribute equal
    to val.
    """
    cursor = tag
    while cursor:
        # lambda f: f.has_attr('class') and 'volanta' in f.get('class')
        attrs = [tag for tag in cursor.find_all(lambda f: f.has_attr(attr) and val in f.get(attr)) if tag] # 'if tag' just in case tag is None
        if attrs:                                                   # NOTE: shouldn't this actually guard against
            return attrs                                            #   accessing something that ain't mapping?
        else:
            cursor = cursor.parent
    else:
        return None

In [4]:
h2s_by_section = [list(section.find_all('h2')) for section in [section for section in soup.find_all('section')]]
for i, l in enumerate(h2s_by_section):
    print(f'<{i}> {len(l)} |', end = ' ')

<0> 6 | <1> 14 | <2> 12 | <3> 6 | <4> 7 | <5> 4 | <6> 5 | <7> 4 | <8> 7 | <9> 6 | <10> 6 | <11> 6 | <12> 4 | <13> 1 | <14> 6 | <15> 4 | <16> 4 | <17> 4 | 

In [5]:
for i, section in enumerate(h2s_by_section):
    for j, h2 in enumerate(section):
        tags_with_hrefs_list = tag_nearest_attrs(h2, 'href')
        print(f'<section> {i} // <h2> in section {j} // #hrefs {len(tags_with_hrefs_list)} >\n',
            f'{h2}\n',
            tags_with_hrefs_list, end = '\n--------------\n')
    print('=' * 80)    

<section> 0 // <h2> in section 0 // #hrefs 1 >
 <h2>Cambios en el Presupuesto 2023: se podrá descontar de Ganancias el 40% de las cuotas de los colegios privados</h2>
 [<a aria-label="/politica/cambios-presupuesto-podra-descontar-ganancias-40-gastos-escolares_0_hwX5mlgd6d.html" class="link_article" href="/politica/cambios-presupuesto-podra-descontar-ganancias-40-gastos-escolares_0_hwX5mlgd6d.html" onclick="metricsView('bloques_home','contenedor_1_piso_tipo_b|hwX5mlgd6d');" target="_self"> </a>]
--------------
<section> 0 // <h2> in section 1 // #hrefs 1 >
 <h2 class="title-rel">El Gobierno busca que los jueces paguen Ganancias y Cambiemos lo rechaza</h2>
 [<a aria-label="/politica/avance-justicia-k-piden-incluir-articulo-presupuesto-jueces-paguen-ganancias_0_50E5hnSIYm.html" href="/politica/avance-justicia-k-piden-incluir-articulo-presupuesto-jueces-paguen-ganancias_0_50E5hnSIYm.html" onclick="metricsView('bloques_home','50E5hnSIYm');" target="_self">
<div class="box-title-rel">
<span 

In [6]:
for i, section in enumerate(h2s_by_section):
    for j, h2 in enumerate(section):
        tags_with_hrefs_list = tag_nearest_attrs(h2, 'href')
        if len(tags_with_hrefs_list) > 1:
            print(f'In <section> {i}, <h2> {i} has', len(tags_with_hrefs_list))


In [7]:
for i, section in enumerate(h2s_by_section):
    print(f'section {i}'); print('-' * len(f'section {i}'))
    for j, h2 in enumerate(section):
        print(f'  h2 {j}>', h2.get_text())
        
        print('\tvolantas:', list(h2.parent.parent.find_all(class_ = 'volanta')) if
                            h2.parent.parent.find_all(class_ = 'volanta') else None)
        print('\tvolanta:', list(h2.parent.parent.find_all(class_ = 'volanta'))[0] if
                            h2.parent.parent.find_all(class_ = 'volanta') else None)
        print('\tvolanta_text:', list(h2.parent.parent.find_all(class_ = 'volanta'))[0].get_text() if
                            h2.parent.parent.find_all(class_ = 'volanta') else None)
        print()

        print('\tauthors:', list(h2.parent.parent.find_all(class_ = 'author')) if
                            h2.parent.parent.find_all(class_ = 'author') else None)
        print('\tauthor:', list(h2.parent.parent.find_all(class_ = 'author'))[0] if
                            h2.parent.parent.find_all(class_ = 'author') else None)
        print('\tauthor:_text', list(h2.parent.parent.find_all(class_ = 'author'))[0].get_text() if
                            h2.parent.parent.find_all(class_ = 'author') else None)
        print()

        print('\tsummarys:', list(h2.parent.parent.find_all(class_ = 'summary')) if
                            h2.parent.parent.find_all(class_ = 'summary') else None)
        print('\tsummary:', list(h2.parent.parent.find_all(class_ = 'summary'))[0] if
                            h2.parent.parent.find_all(class_ = 'summary') else None)
        print('\tsummary_text:', list(h2.parent.parent.find_all(class_ = 'summary'))[0].get_text() if
                            h2.parent.parent.find_all(class_ = 'summary') else None)
        print()                            
        print('=' * 80)


section 0
---------
  h2 0> Cambios en el Presupuesto 2023: se podrá descontar de Ganancias el 40% de las cuotas de los colegios privados
	volantas: [<p class="volanta">Impactará en la clase media</p>]
	volanta: <p class="volanta">Impactará en la clase media</p>
	volanta_text: Impactará en la clase media

	authors: [<p class="author">Gustavo Berón</p>]
	author: <p class="author">Gustavo Berón</p>
	author:_text Gustavo Berón

	summarys: [<p class="summary">Fue uno de los pedidos de la oposición para que el proyecto avance en Diputados y sea tratado el próximo martes. Las demás modificaciones concedidas por el oficialismo. </p>]
	summary: <p class="summary">Fue uno de los pedidos de la oposición para que el proyecto avance en Diputados y sea tratado el próximo martes. Las demás modificaciones concedidas por el oficialismo. </p>
	summary_text: Fue uno de los pedidos de la oposición para que el proyecto avance en Diputados y sea tratado el próximo martes. Las demás modificaciones concedida

In [8]:
for i, section in enumerate(h2s_by_section):
    print(f'section {i}')
    print('-' * len(f'section {i}'))
    for j, h2 in enumerate(section):
        class_volantas = tag_nearest_attr_val(h2, 'class', 'volanta')
        class_authors = tag_nearest_attr_val(h2, 'class', 'author')
        class_summarys = tag_nearest_attr_val(h2, 'class', 'summary')
        
        print(f'  h2 {j}>')
        print(f'type(class_volantas) = {type(class_volantas)} //'
              f'type(class_authors) = {type(class_authors)} //'
              f'type(class_summarys) = {type(class_summarys)}')
        
        # print('\tvolantas:', list(h2.parent.parent.find_all(class_ = 'volanta')) if
        #                     h2.parent.parent.find_all(class_ = 'volanta') else None)
        # print('\tvolanta:', list(h2.parent.parent.find_all(class_ = 'volanta'))[0] if
        #                     h2.parent.parent.find_all(class_ = 'volanta') else None)
        # print('\tvolanta_text:', list(h2.parent.parent.find_all(class_ = 'volanta'))[0].get_text() if
        #                     h2.parent.parent.find_all(class_ = 'volanta') else None)
        # print()
# 
        # print('\tauthors:', list(h2.parent.parent.find_all(class_ = 'author')) if
        #                     h2.parent.parent.find_all(class_ = 'author') else None)
        # print('\tauthor:', list(h2.parent.parent.find_all(class_ = 'author'))[0] if
        #                     h2.parent.parent.find_all(class_ = 'author') else None)
        # print('\tauthor:_text', list(h2.parent.parent.find_all(class_ = 'author'))[0].get_text() if
        #                     h2.parent.parent.find_all(class_ = 'author') else None)
        # print()
# 
        # print('\tsummarys:', list(h2.parent.parent.find_all(class_ = 'summary')) if
        #                     h2.parent.parent.find_all(class_ = 'summary') else None)
        # print('\tsummary:', list(h2.parent.parent.find_all(class_ = 'summary'))[0] if
        #                     h2.parent.parent.find_all(class_ = 'summary') else None)
        # print('\tsummary_text:', list(h2.parent.parent.find_all(class_ = 'summary'))[0].get_text() if
        #                     h2.parent.parent.find_all(class_ = 'summary') else None)
        # print()                            
        print('=' * 80)

section 0
---------
  h2 0>
type(class_volantas) = <class 'list'> //type(class_authors) = <class 'list'> //type(class_summarys) = <class 'list'>
  h2 1>
type(class_volantas) = <class 'list'> //type(class_authors) = <class 'list'> //type(class_summarys) = <class 'list'>
  h2 2>
type(class_volantas) = <class 'list'> //type(class_authors) = <class 'list'> //type(class_summarys) = <class 'list'>
  h2 3>
type(class_volantas) = <class 'list'> //type(class_authors) = <class 'list'> //type(class_summarys) = <class 'list'>
  h2 4>
type(class_volantas) = <class 'list'> //type(class_authors) = <class 'list'> //type(class_summarys) = <class 'list'>
  h2 5>
type(class_volantas) = <class 'list'> //type(class_authors) = <class 'list'> //type(class_summarys) = <class 'list'>
section 1
---------
  h2 0>
type(class_volantas) = <class 'list'> //type(class_authors) = <class 'list'> //type(class_summarys) = <class 'list'>
  h2 1>
type(class_volantas) = <class 'list'> //type(class_authors) = <class 'list'> 

In [9]:
def tag_nearest_name(tag, name):
    cursor = tag
    while cursor:
        if cursor.name == name:
            return cursor
        else:
            cursor = cursor.parent
    else:
        return None

In [10]:
def tag_nearest_attr_val(tag, attr, val):
    """
    Given a tag, walks upwards the hierarchy and stops when reaching a tag
    with at least one tag in its descendants hierarchy having attr attribute
    with value val, and returns a list of the nodes with that attribute equal
    to val.
    """
    cursor = tag
    while cursor:
        # lambda f: f.has_attr('class') and 'volanta' in f.get('class')
        attrs = [tag for tag in cursor.find_all(lambda f: f.has_attr(attr) and val in f.get(attr)) if tag] # 'if tag' just in case tag is None
        if attrs:                                                   # NOTE: shouldn't this actually guard against
            return attrs                                            #   accessing something that ain't mapping?
        else:
            cursor = cursor.parent
    else:
        return None

In [11]:
for i, section in enumerate(h2s_by_section):
    for j, h2 in enumerate(section):
        print(f'<section> [{i}] // h2 [{j}]'); print(len(f'<section> [{i}] // h2 [{j}]') * '-')
        print(tag_nearest_name(h2, 'article').prettify())
        print(tag_nearest_name(h2, 'article').prettify())

<section> [0] // h2 [0]
-----------------------
<article class="content-nota list-format twoxone_no_foto">
 <div class="mt">
  <div class="max-h flex-change">
   <h2>
    Cambios en el Presupuesto 2023: se podrá descontar de Ganancias el 40% de las cuotas de los colegios privados
   </h2>
   <p class="volanta">
    Impactará en la clase media
   </p>
  </div>
  <div class="data-txt">
   <p class="author">
    Gustavo Berón
   </p>
  </div>
  <p class="summary">
   Fue uno de los pedidos de la oposición para que el proyecto avance en Diputados y sea tratado el próximo martes. Las demás modificaciones concedidas por el oficialismo.
  </p>
 </div>
 <a aria-label="/politica/cambios-presupuesto-podra-descontar-ganancias-40-gastos-escolares_0_hwX5mlgd6d.html" class="link_article" href="/politica/cambios-presupuesto-podra-descontar-ganancias-40-gastos-escolares_0_hwX5mlgd6d.html" onclick="metricsView('bloques_home','contenedor_1_piso_tipo_b|hwX5mlgd6d');" target="_self">
 </a>
</article>

<ar

In [12]:
# failed experiment for harvesting a volanta.
for i, section in enumerate(h2s_by_section):
    for j, h2 in enumerate(section):
        print(f'<section>[{i}] | <h2>[{j}]>')
        volanta = tag_nearest_attr_val(h2, 'class', 'volanta')
        # the problem here is that if there's no volanta associated with the <h2> being processed
        # another one's gonna be picked...
        if volanta:
            print("\tvolanta_text:", volanta[0].get_text())
        print("\th2_Text:", h2.get_text())
        if volanta:    
            print(f"\tText as seen by consumer:\n\t\t {volanta[0].get_text()} {h2.get_text()}")
        summary = tag_nearest_attr_val(h2, 'class', 'summary')
        if summary:
            print("\tsummary_text:", summary[0].get_text())
    print("-"*80)

<section>[0] | <h2>[0]>
	volanta_text: Impactará en la clase media
	h2_Text: Cambios en el Presupuesto 2023: se podrá descontar de Ganancias el 40% de las cuotas de los colegios privados
	Text as seen by consumer:
		 Impactará en la clase media Cambios en el Presupuesto 2023: se podrá descontar de Ganancias el 40% de las cuotas de los colegios privados
	summary_text: Fue uno de los pedidos de la oposición para que el proyecto avance en Diputados y sea tratado el próximo martes. Las demás modificaciones concedidas por el oficialismo. 
<section>[0] | <h2>[1]>
	volanta_text: Impactará en la clase media
	h2_Text: El Gobierno busca que los jueces paguen Ganancias y Cambiemos lo rechaza
	Text as seen by consumer:
		 Impactará en la clase media El Gobierno busca que los jueces paguen Ganancias y Cambiemos lo rechaza
	summary_text: Fue uno de los pedidos de la oposición para que el proyecto avance en Diputados y sea tratado el próximo martes. Las demás modificaciones concedidas por el oficiali

In [24]:
print(h2s_by_section[1][7].parent.parent.parent.prettify())

<li aria-label="/policiales/femicidio-jujuy-autopsia-revelo-murio-chica-16-anos-cuyo-novio-imputado_0_qkwTWoNQed.html" style="cursor: pointer;">
 <a href="/policiales/femicidio-jujuy-autopsia-revelo-murio-chica-16-anos-cuyo-novio-imputado_0_qkwTWoNQed.html" onclick="metricsView('bloques_home','bloque_ultimo_momento|qkwTWoNQed', true);">
  <article class="flex-change">
   <h2>
    Femicidio en Jujuy: la autopsia reveló cómo murió la chica de 16 años, cuyo novio fue imputado
   </h2>
   <span>
    05:03
   </span>
  </article>
 </a>
</li>



In [28]:

for i, section in enumerate(h2s_by_section):
    for j, h2 in enumerate(section):
        h2_parents_names = [tag.name for tag in h2.parents]
        h2_article = list(h2.parents)[h2_parents_names.index('article')]
        
        print(f'<section>[{i}] h2[{j}]'); print(len(f'<section>[{i}] h2[{j}]') * '=')
        
        print("Relevant extracts:"); print('-' * len("Relevant extracts:"))

        
        title_text = h2.get_text()
        print("Title: ", title_text)  # warning... you're vacilating. Don't consider h2's children... only h2 itself
        
        title_word_count = len(title_text.split(' '))
        print("Title Word Count: ", title_word_count)
        
        h2_author_tag = h2_article.find(class_ = 'author') or h2_article.find(class_ = 'author-rel')
        try: 
            author_text = h2_author_tag.get_text() 
        except AttributeError: 
            author_text = None
        print("Author Text: ", author_text)

        summary_tag = h2_article.find(class_ = 'summary') or h2_article.find(class_ = 'summary_rel')
        try:
            summary_text = summary_tag.get_text()
        except:
            summary_text = None
        print("Summary: ", summary_text)

        volanta_tag = h2_article.find(class_ = 'volanta')
        try:
            volanta_text = volanta_tag.get_text()
        except AttributeError:
            volanta_text = None
        print("Volanta: ", volanta_text)

        h2_hrefs_list = list(h2_article.find_all(href = True))
        
        # if h2_hrefs_list is empty, try parent node:
        #   this happens in the 'ULTIMO MOMENTO' panel, where areas are anchored by enclosing the whole article
        if not h2_hrefs_list:
            h2_hrefs_list = list(h2_article.parent.parent.find_all(href = True))

        h2_main_link = h2_hrefs_list[0]['href'] if h2_hrefs_list else None

        slug = h2_main_link if h2_main_link and h2_main_link[0] == '/' else None
        category = slug.split('/')[1] if slug else None
        
        print("Main Link: ", h2_main_link)
        print("Slug: ", slug)
        # If despite having a link, there's no category, try something different
        if h2_main_link and h2_main_link.split('/')[2] in ('www.clarin.com', 'www.ole.com.ar', 'elle.clarin.com' ) and not category:   
            print('+++ DIAGNOSTIC: ', h2_main_link.split('/')[3])
        print("Category: ", category)

        print("Origen: ", "Clarin")
        print('-' * 120 + '\n')
    print('=' * 80)
            
        

<section>[0] h2[0]
Relevant extracts:
------------------
Title:  Cambios en el Presupuesto 2023: se podrá descontar de Ganancias el 40% de las cuotas de los colegios privados
Title Word Count:  19
Author Text:  Gustavo Berón
Summary:  Fue uno de los pedidos de la oposición para que el proyecto avance en Diputados y sea tratado el próximo martes. Las demás modificaciones concedidas por el oficialismo. 
Volanta:  Impactará en la clase media
Main Link:  /politica/cambios-presupuesto-podra-descontar-ganancias-40-gastos-escolares_0_hwX5mlgd6d.html
Slug:  /politica/cambios-presupuesto-podra-descontar-ganancias-40-gastos-escolares_0_hwX5mlgd6d.html
Category:  politica
Origen:  Clarin
------------------------------------------------------------------------------------------------------------------------

<section>[0] h2[1]
Relevant extracts:
------------------
Title:  El Gobierno busca que los jueces paguen Ganancias y Cambiemos lo rechaza
Title Word Count:  12
Author Text:  Gustavo Berón
Summ